<a href="https://colab.research.google.com/github/Befezdow/ai_test/blob/master/course_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

drive.mount('/content/gdrive')
names_array = ['gender', 'age', 'class1', 'class2', 'class3', *['t{}'.format(i + 1) for i in range(60000)]]
url = '/content/gdrive/My Drive/ML/half.csv'
data = pd.read_csv(url, names=names_array)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
data = data.fillna(data.median(axis = 0), axis = 0)
data

In [0]:
current_count_per_sensor = 5000
needed_count_per_sensor = 100
sensors_count = 12

non_sliceable_columns = data.iloc[:,0:5]
sliceable_columns = data.iloc[:,5:]

current_offset = 0
new_data = pd.DataFrame()
for i in range(int(np.floor(current_count_per_sensor / needed_count_per_sensor))):
  new_offset = current_offset + needed_count_per_sensor
  new_slice = pd.DataFrame()
  
  for j in range(sensors_count):
    index_from = j * current_count_per_sensor + current_offset
    index_to = j * current_count_per_sensor + new_offset
    data_slice = sliceable_columns.iloc[:, index_from:index_to]
    new_slice = pd.concat([new_slice, data_slice], axis=1, sort=False, ignore_index=True)
  
  completed_rows = pd.concat([non_sliceable_columns.copy(), new_slice], axis=1, sort=False, ignore_index=True)
  new_data = pd.concat([new_data, completed_rows], axis=0, sort=False, ignore_index=True)
  current_offset += needed_count_per_sensor

new_data.columns = ['gender', 'age', 'class1', 'class2', 'class3', *['t{}'.format(i + 1) for i in range(needed_count_per_sensor * sensors_count)]]

In [0]:
data = new_data

In [0]:
# убираем лишние классовые признаки
data = data.drop('class2', axis = 1)
data = data.drop('class3', axis = 1)

#собираем числовые колонки
numerical_columns = [c for c in data.columns if data[c].dtype.name != 'object' and c != 'class1']

# векторизируем колонку gender
data.at[data['gender'] == 'Male', 'gender'] = 0
data.at[data['gender'] == 'Female', 'gender'] = 1

# нормализуем числовые атрибуты
data_numerical = data[numerical_columns]
data_numerical = (data_numerical - data_numerical.mean(axis = 0))/data_numerical.std(axis = 0)
data[numerical_columns] = data_numerical

data

In [0]:
# бьем данные на входы и выходы
x = data.drop('class1', axis = 1)
y = data['class1']

In [0]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

n_train, _ = x_train.shape 
n_test,  _ = x_test.shape 

print('Train dataset size: {}'.format(n_train))
print('Test dataset size: {}'.format(n_test))

In [0]:
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [0]:
import torch

x_train = torch.Tensor(x_train)
y_train = torch.Tensor(y_train).long() - 1

x_test = torch.Tensor(x_test)
y_test = torch.Tensor(y_test).long() - 1

print('Train X shape: {}'.format(x_train.shape))
print('Train Y shape: {}'.format(y_train.shape))

print('Test X shape: {}'.format(x_test.shape))
print('Test Y shape: {}'.format(y_test.shape))

In [0]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(1202, 500)
    self.fc2 = nn.Linear(500, 500)
    self.fc3 = nn.Linear(500, 9)
  
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return F.log_softmax(x, dim=1)

net = Net().double()

# Осуществляем оптимизацию путем стохастического градиентного спуска
# optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
# Создаем функцию потерь
criterion = nn.NLLLoss()

epochs_count = 10
batch_size = 100

dataset_size = x_train.shape[0]
batches_count = int(np.ceil(dataset_size / batch_size))

for epoch_index in range(epochs_count):
  for batch_index in range(batches_count):
    x_var = Variable(x_train[batch_index * batch_size:batch_index * batch_size + batch_size])
    y_var = Variable(y_train[batch_index * batch_size:batch_index * batch_size + batch_size])

    optimizer.zero_grad()
    net_out = net(x_var.double())
    loss = criterion(net_out, y_var)
    loss.backward()
    optimizer.step()
    
    # print('Train Epoch: {}; Batch: {}/{}; tLoss: {:.6f}'.format(epoch_index + 1, batch_index + 1, batches_count, loss.data.item()))


In [0]:
dataset_size = x_test.shape[0]

batch_size = 100
batches_count = int(np.ceil(dataset_size / batch_size))

test_loss = 0
correct = 0
for batch_index in range(batches_count):
    x_var = Variable(x_test[batch_index * batch_size:batch_index * batch_size + batch_size])
    y_var = Variable(y_test[batch_index * batch_size:batch_index * batch_size + batch_size])

    net_out = net(x_var.double())
    test_loss += criterion(net_out, y_var).item()  # суммируем потери со всех партий
    pred = net_out.data.max(1)[1]  # получаем индекс максимального значения
    correct += pred.eq(y_var.data).sum()

test_loss /= dataset_size
print('Average loss: {:.4f}; Accuracy: {}/{} ({:.2f}%)'.format(
    test_loss, correct, dataset_size, 100. * correct / dataset_size
))

In [0]:
train_error_list = []
test_error_list = []
nodes_count_list = []

nodes_count_from = 200
nodes_count_to = 1000
delta = 100

for nodes_count in np.arange(nodes_count_from, nodes_count_to + 1, delta):
  print('Nodes count: {}'.format(nodes_count))
  class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()
      self.fc1 = nn.Linear(1202, nodes_count)
      self.fc2 = nn.Linear(nodes_count, nodes_count)
      self.fc3 = nn.Linear(nodes_count, 9)
    
    def forward(self, x):
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return F.log_softmax(x, dim=1)

  net = Net().double()

  # Осуществляем оптимизацию путем стохастического градиентного спуска
  # optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
  optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
  # Создаем функцию потерь
  criterion = nn.NLLLoss()

  epochs_count = 10
  batch_size = 100

  dataset_size = x_test.shape[0]
  batches_count = int(np.ceil(dataset_size / batch_size))

  for epoch_index in range(epochs_count):
    train_loss = 0
    correct = 0
    for batch_index in range(batches_count):
      x_var = Variable(x_train[batch_index * batch_size:batch_index * batch_size + batch_size])
      y_var = Variable(y_train[batch_index * batch_size:batch_index * batch_size + batch_size])

      optimizer.zero_grad()
      net_out = net(x_var.double())
      loss = criterion(net_out, y_var)

      train_loss += loss.item()
      pred = net_out.data.max(1)[1]  # получаем индекс максимального значения
      correct += pred.eq(y_var.data).sum()

      loss.backward()
      optimizer.step()

  train_loss /= dataset_size
  train_percentage = 100. * correct.item() / dataset_size
  print('Train :: Average loss: {:.4f}; Accuracy: {}/{} ({:.2f}%)'.format(train_loss, correct, dataset_size, train_percentage))


  dataset_size = x_test.shape[0]
  batch_size = 100
  batches_count = int(np.ceil(dataset_size / batch_size))

  test_loss = 0
  correct = 0
  for batch_index in range(batches_count):
      x_var = Variable(x_test[batch_index * batch_size:batch_index * batch_size + batch_size])
      y_var = Variable(y_test[batch_index * batch_size:batch_index * batch_size + batch_size])

      net_out = net(x_var.double())
      test_loss += criterion(net_out, y_var).item()  # суммируем потери со всех партий
      pred = net_out.data.max(1)[1]  # получаем индекс максимального значения
      correct += pred.eq(y_var.data).sum()

  test_loss /= dataset_size
  test_percentage = 100. * correct.item() / dataset_size
  print('Test :: Average loss: {:.4f}; Accuracy: {}/{} ({:.2f}%)'.format(
      test_loss, correct, dataset_size, test_percentage
  ))

  train_error_list.append(100 - train_percentage)
  test_error_list.append(100 - test_percentage)
  nodes_count_list.append(nodes_count)

In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize = (15, 8))
plt.plot(nodes_count_list, train_error_list, label = 'Train', color = 'blue')
plt.plot(nodes_count_list, test_error_list, label = 'Test', color = 'red')

plt.xlabel('Hidden layers size')
plt.ylabel('Error')
plt.title('Size vs Error')
plt.legend()
plt.grid()

In [0]:
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()
      self.fc1 = nn.Linear(1202, 400)
      self.fc2 = nn.Linear(400, 400)
      self.fc3 = nn.Linear(400, 9)
    
    def forward(self, x):
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return F.log_softmax(x, dim=1)

weight_decays = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8]
train_error_list = []
test_error_list = []

for decay in weight_decays:
  print('Weight decay: {}'.format(decay))
  net = Net().double()
  optimizer = torch.optim.Adam(net.parameters(), lr=0.001, weight_decay=decay)
  criterion = nn.NLLLoss()

  epochs_count = 10
  batch_size = 100

  dataset_size = x_test.shape[0]
  batches_count = int(np.ceil(dataset_size / batch_size))

  for epoch_index in range(epochs_count):
    train_loss = 0
    correct = 0
    for batch_index in range(batches_count):
      x_var = Variable(x_train[batch_index * batch_size:batch_index * batch_size + batch_size])
      y_var = Variable(y_train[batch_index * batch_size:batch_index * batch_size + batch_size])

      optimizer.zero_grad()
      net_out = net(x_var.double())
      loss = criterion(net_out, y_var)

      train_loss += loss.item()
      pred = net_out.data.max(1)[1]  # получаем индекс максимального значения
      correct += pred.eq(y_var.data).sum()

      loss.backward()
      optimizer.step()

  train_loss /= dataset_size
  train_percentage = 100. * correct.item() / dataset_size
  print('Train :: Average loss: {:.4f}; Accuracy: {}/{} ({:.2f}%)'.format(train_loss, correct, dataset_size, train_percentage))


  dataset_size = x_test.shape[0]
  batch_size = 100
  batches_count = int(np.ceil(dataset_size / batch_size))

  test_loss = 0
  correct = 0
  for batch_index in range(batches_count):
      x_var = Variable(x_test[batch_index * batch_size:batch_index * batch_size + batch_size])
      y_var = Variable(y_test[batch_index * batch_size:batch_index * batch_size + batch_size])

      net_out = net(x_var.double())
      test_loss += criterion(net_out, y_var).item()  # суммируем потери со всех партий
      pred = net_out.data.max(1)[1]  # получаем индекс максимального значения
      correct += pred.eq(y_var.data).sum()

  test_loss /= dataset_size
  test_percentage = 100. * correct.item() / dataset_size
  print('Test :: Average loss: {:.4f}; Accuracy: {}/{} ({:.2f}%)'.format(
      test_loss, correct, dataset_size, test_percentage
  ))

  train_error_list.append(100 - train_percentage)
  test_error_list.append(100 - test_percentage)


In [0]:
train_error_list = train_error_list[-8:]
test_error_list = test_error_list[-8:]

In [0]:
plt.figure(figsize = (15, 8))
plt.plot(weight_decays, train_error_list, label = 'Train', color = 'blue')
plt.plot(weight_decays, test_error_list, label = 'Test', color = 'red')

plt.xlabel('Weight decay')
plt.ylabel('Error')
plt.title('Weight decay vs Error')
plt.legend()
plt.grid()